In [1]:
import pandas as pd
import nltk
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

/home/sc0001/Documents/SoftRec/sofRec/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
review_meta = pd.read_csv("../../data/review_metadata.csv")

/tmp/ipykernel_98567/145584005.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  review_meta = pd.read_csv("../../data/review_metadata.csv")


In [6]:
review_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145863 entries, 0 to 145862
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         145863 non-null  int64  
 1   overall            145863 non-null  float64
 2   verified           145863 non-null  bool   
 3   reviewTime         145863 non-null  object 
 4   reviewerID         145863 non-null  object 
 5   asin               145863 non-null  object 
 6   reviewerName       145836 non-null  object 
 7   reviewText         145834 non-null  object 
 8   summary            145844 non-null  object 
 9   vote               30493 non-null   object 
 10  category           145863 non-null  object 
 11  tech1              26 non-null      object 
 12  description        145863 non-null  object 
 13  fit                0 non-null       float64
 14  title              145863 non-null  object 
 15  also_buy           145863 non-null  object 
 16  te

In [2]:
def preprocess_text(text):
    # Tokenize the text
    text = str(text)
    tokens = word_tokenize(text.lower())
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [5]:
review_meta['title'] = review_meta['title'].apply(preprocess_text)
review_meta['summary'] = review_meta['summary'].apply(preprocess_text)
review_meta['description'] = review_meta['description'].apply(preprocess_text)
review_meta['category'] = review_meta['category'].apply(preprocess_text)

In [6]:
review_meta.to_csv("../../data/review_metadata_with_embeddings.csv")

/home/sc0001/Documents/SoftRec/sofRec/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Generate Embeddings for text features

In [3]:
name = 'bert-base-nli-mean-tokens'
model = SentenceTransformer(name)

In [4]:
def get_embeddings(text):
    sentences = list(text)
    return model.encode(sentences)

In [9]:
review_meta['summary_embeddings'] = review_meta['summary'].apply(lambda x: get_embeddings(x))
review_meta.to_csv("../../data/review_metadata_with_embeddings.csv")

In [10]:
review_meta['title_embeddings'] = review_meta['title'].apply(lambda x: get_embeddings(x))
review_meta.to_csv("../../data/review_metadata_with_embeddings.csv")

In [38]:
review_meta = pd.read_csv("../../data/review_metadata_with_embeddings.csv")
review_meta = review_meta.drop(columns=["Unnamed: 0"])
review_meta.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote,category,...,similar_item,date,price,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost,summary_embeddings,title_embeddings
0,5.0,False,"07 23, 2008",A8IOST6U6WH9B,0615179088,C. Radey,Human Japanese is a truly superb introduction ...,human japanese,12,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...
1,5.0,False,"06 4, 2008",A1MUV9F35OROS5,0615179088,D. Abel,I got Human Japanese as a demo from its websit...,best japanese program available,11,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.07393144 -0.05106744 2.1081572 ... 0.6...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...
2,4.0,False,"04 8, 2008",A27PAMABWVQ892,0615179088,piepiepie75,My first experience with Human Japanese was th...,better human japanese 1 ... much changed .,99,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.07393144 -0.05106744 2.1081572 ... 0.6...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...
3,5.0,False,"03 26, 2008",A3HWWVK0L3JEKF,0615179088,K. Grier,This is the first language software that I hav...,great product,4,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[-0.39693055 -0.31397188 2.0098865 ... 0.1...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...
4,5.0,False,"02 20, 2008",A3NO2V2JU4Y8UY,0615179088,H. Granat,Human japanese is the best pc program for lear...,love !,2,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[-0.21905293 -0.86112505 1.9966931 ... -0.0...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...


<br> Realized that the description and asin were unique for each software, hence we have only 650 distinct softwares.

In [39]:

review_meta_1 = review_meta.groupby('asin').agg({'description':'first'}).reset_index()

In [40]:
review_meta_1['description_embeddings'] = review_meta_1['description'].apply(lambda x: get_embeddings(x))

In [20]:
review_meta_1.head()

,asin,description,description_embeddings
0,007742817X,"[ 'slides video , teaching public speaking onl...","[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
1,0439740207,"[ 'teaches child fun way type ' , `` , `` ]","[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
2,043983595X,[ `` 50 activity developing essential 1st & am...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
3,0615179088,[ ' < div > human japanese software window pc ...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
4,061591067X,[ 'learn japanese long-awaited sequel human ja...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."


In [41]:
review_meta =  review_meta.merge(review_meta_1, on='asin', how='inner')

In [42]:
review_meta = review_meta.drop(columns=['description_y'])

In [43]:
review_meta.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote,category,...,date,price,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost,summary_embeddings,title_embeddings,description_embeddings
0,5.0,False,"07 23, 2008",A8IOST6U6WH9B,0615179088,C. Radey,Human Japanese is a truly superb introduction ...,human japanese,12,"[ 'software ' , 'education & reference ' , 'la...",...,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
1,5.0,False,"06 4, 2008",A1MUV9F35OROS5,0615179088,D. Abel,I got Human Japanese as a demo from its websit...,best japanese program available,11,"[ 'software ' , 'education & reference ' , 'la...",...,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.07393144 -0.05106744 2.1081572 ... 0.6...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
2,4.0,False,"04 8, 2008",A27PAMABWVQ892,0615179088,piepiepie75,My first experience with Human Japanese was th...,better human japanese 1 ... much changed .,99,"[ 'software ' , 'education & reference ' , 'la...",...,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.07393144 -0.05106744 2.1081572 ... 0.6...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
3,5.0,False,"03 26, 2008",A3HWWVK0L3JEKF,0615179088,K. Grier,This is the first language software that I hav...,great product,4,"[ 'software ' , 'education & reference ' , 'la...",...,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[-0.39693055 -0.31397188 2.0098865 ... 0.1...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."
4,5.0,False,"02 20, 2008",A3NO2V2JU4Y8UY,0615179088,H. Granat,Human japanese is the best pc program for lear...,love !,2,"[ 'software ' , 'education & reference ' , 'la...",...,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994,[[-0.21905293 -0.86112505 1.9966931 ... -0.0...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576..."


In [44]:
review_meta = review_meta[review_meta.verified == True]

In [45]:
len(review_meta)

15456

In [46]:
review_meta.dropna(subset = ['reviewText'], inplace=True)

In [48]:
review_meta.overall.value_counts()

overall
5.0    8204
4.0    2625
1.0    2390
3.0    1349
2.0     887
Name: count, dtype: int64

### 

In [49]:
len(review_meta)

15455

### Correct rating with Review Text
We use kmeans to cluster the tfidvector of all reviewtext to correct the ratings. 

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(review_meta['reviewText']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

(15455, 16724)


In [81]:
from sklearn.cluster import KMeans
km = KMeans(5)
km.fit(tfidf_matrix)

KMeans(n_clusters=5)

In [82]:
ls_val = km.labels_.tolist()

In [86]:
clusters= { 'cluster_1' : ls_val.count(0),
           'cluster_2' : ls_val.count(1),
           'cluster_3' : ls_val.count(2),
           'cluster_4' : ls_val.count(3),
           'cluster_5' : ls_val.count(4),
}

In [87]:
clusters

{'cluster_1': 7596,
 'cluster_2': 6181,
 'cluster_3': 463,
 'cluster_4': 291,
 'cluster_5': 924}

In [88]:
# Add kmeans clusters
review_meta['clusters'] = ls_val

#### For each cluster we count the numbe rof positives and negatives in them the use to encode ratings 1 - 5

In [89]:
from transformers import pipeline
import pandas as pd

# Output labels
candidate_labels = ["positive", "negative"]

# device=0 for GPU usage
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)


def count_predictions(df):
    predictions = []
    # Iterate over each row in the DataFrame
    for i in range(len(df)):
        input_text = df['reviewText'].iloc[i]

        # Perform zero-shot classification
        model_dict = classifier(input_text, candidate_labels, multi_label=True)

        # Zip results to dict
        result_dict = dict(zip(model_dict['labels'], model_dict['scores']))

        # Assign the highest scoring label as the predicted label
        predicted_label = max(result_dict, key=result_dict.get)
        predictions.append(predicted_label)

    return {"postive":predictions.count('positive'), "negative":predictions.count('negative')}
    

In [90]:
## For cluster 1
count_predictions(review_meta[review_meta.clusters == 0])

/home/sc0001/Documents/SoftRec/sofRec/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'postive': 5334, 'negative': 2262}

In [99]:
5334 / (5334 + 2262)

0.7022116903633492

In [91]:
## For cluster 2
count_predictions(review_meta[review_meta.clusters == 1])

{'postive': 3638, 'negative': 2543}

In [95]:
3638 / (3638 + 2543)

0.5885779000161786

In [92]:
## For cluster 3
count_predictions(review_meta[review_meta.clusters == 2])

{'postive': 462, 'negative': 1}

In [96]:
462 / (462 +1)

0.9978401727861771

In [93]:
## For cluster 4
count_predictions(review_meta[review_meta.clusters == 3])

{'postive': 286, 'negative': 5}

In [97]:
286 / (286 + 5)

0.9828178694158075

In [94]:
## For cluster 5
count_predictions(review_meta[review_meta.clusters == 4])

{'postive': 861, 'negative': 63}

In [98]:
861 / (861 + 63)

0.9318181818181818

##### We assigned ratings based on the group with the realtive highest positive sentiments

In [100]:
rating_map = {0:4,
 1:5,
 2:1,
 3:2,
 4:3}

In [102]:
review_meta['ratings'] = review_meta['clusters'].map(rating_map)

In [103]:
review_meta.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote,category,...,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost,summary_embeddings,title_embeddings,description_embeddings,clusters,ratings
8,5.0,True,"12 2, 2014",A2Q3KBSVA7F56Z,0615179088,Alvin Truthfinder,Very easy product to use. It's clear. Concise....,five star,NaN,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,Education & Reference,0.008,19.97,3.994,[[-0.16857387 -0.5855289 2.4885037 ... 0.2...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576...",4,3
9,5.0,True,"11 14, 2014",A32OSD6FHVIK4I,0615179088,Mandy Wells,I love the Human Japanese software. I had a li...,love !,NaN,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,Education & Reference,0.008,19.97,3.994,[[-0.21905293 -0.86112505 1.9966931 ... -0.0...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576...",1,5
10,3.0,True,"11 8, 2014",A37RK6IW147989,0615179088,Wendy Lee Riggsbee,"only problem, daughter lost a code and ow cann...",three star,NaN,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,Education & Reference,0.008,19.97,3.994,[[ 0.24971181 -0.2821653 2.1901054 ... 0.1...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576...",0,4
11,4.0,True,"09 22, 2014",A12ZEYNVT3PCAG,0615179088,Ean P.,"For an intro to learning Japanese, I found it ...",say satisfied overall presentation,NaN,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,Education & Reference,0.008,19.97,3.994,[[-0.05452485 -0.14933898 2.2738338 ... 0.2...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576...",1,5
12,3.0,True,"09 5, 2014",A3VCAL31C8EBRM,0615179088,KG,There is a lot of great information in here bu...,review,NaN,"[ 'software ' , 'education & reference ' , 'la...",...,NaN,Education & Reference,0.008,19.97,3.994,[[-0.01089804 -0.38519934 2.1462338 ... 0.0...,[[ 0.02127007 -0.17556548 2.008087 ... 0.1...,"[[0.09625992, -0.18804407, 2.2306604, 0.070576...",0,4


In [111]:
review_meta.description.head()

8     [ ' < div > human japanese software window pc ...
9     [ ' < div > human japanese software window pc ...
10    [ ' < div > human japanese software window pc ...
11    [ ' < div > human japanese software window pc ...
12    [ ' < div > human japanese software window pc ...
Name: description, dtype: object

In [108]:
review_meta.rename(columns={"description_x":"description"}, inplace=True)

In [110]:
software_data = review_meta[['asin','brand','price','description','Licensing_Fee','Implemention_cost','Maintenance_cost','summary_embeddings',
       'title_embeddings', 'description_embeddings']]

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

 
def rec_softwares(software_description, software_data):
    software_data['software_description'] = software_description
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(software_data[['software_description','description']])
    
    cosine_sim = cosine_similarity(X)
    target_item_index = 0 
    scores = list(enumerate(cosine_sim[target_item_index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_n = 5
    top_recommendations = scores[0:top_n+1]
    for ind, sore in top_recommendations:
        print(software_data.iloc[ind,:])

In [153]:
rec_softwares("HR program for windows and mac for japanese humans",software_data)

asin                                                             0615179088
brand                                                         Brak Software
price                                                                 39.94
description               [ ' < div > human japanese software window pc ...
Licensing_Fee                                                         0.008
Implemention_cost                                                     19.97
Maintenance_cost                                                      3.994
summary_embeddings        [[-0.16857387 -0.5855289   2.4885037  ...  0.2...
title_embeddings          [[ 0.02127007 -0.17556548  2.008087   ...  0.1...
description_embeddings    [[0.09625992, -0.18804407, 2.2306604, 0.070576...
software_description      HR program for windows and mac for japanese hu...
Name: 8, dtype: object
asin                                                             0615179088
brand                                                         Bra

/tmp/ipykernel_213570/370724902.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  software_data['software_description'] = software_description
